<a href="https://colab.research.google.com/github/Lfahima/datasci_2_manipulation/blob/main/HHA507Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
import polars as pl

In [3]:
import numpy as np

In [4]:
!pip install -U ipykernel
!pip install modin[dask]
# Getting modin to work on colab

In [5]:
import modin.pandas as mpd

In [6]:
import time

## Loading Data Set




I loaded the data set provided by professor Hants into a variable called HCD_df

In [7]:
HCD_df = pd.read_csv('healthcare_data_cleaning.csv')
HCD_df


,Patient Age,Gender,City of Residence,State of Residence,Has Insurance,Visited Last Month,Payment Method,Preferred Doctor,Disease Diagnosed,Medication Prescribed,Type of Appointment,Average Heart Rate,Average BP,Height (in cm),Weight (in kg),Payment Due ($),Last Visit (days ago),Visit Duration (mins),Number of Tests,Prescription Cost ($)
0,45,Other,West Brian,Kentucky,Yes,Yes,Insurance,Dr. Williams,Flu,Med_E,General,66,111,183,70,65.45721578126224,193,92,5,15.71751735532889
1,25,Female,East Jocelynfurt,Colorado,No,Yes,Card,Dr. Johnson,Flu,Med_B,Specialist,59,missing,174,58,430.68365678679174,195,missing,4,80.78647284463952
2,51,Other,South Lindseyland,Ohio,Yes,No,Cash,Dr. Williams,Covid-19,Med_D,General,79,119,161,56,315.0709305262176,missing,37,2,64.3921393009105
3,18,Female,Taylorfort,Pennsylvania,Yes,missing,Cash,Dr. Williams,Flu,Med_A,General,99,115,171,52,320.2998987723972,175,108,2,8.872859215315316
4,48,Female,Birdmouth,Montana,No,No,Insurance,Dr. Brown,NaN,Med_C,Specialist,93,89,186,87,264.2147372473319,188,113,3,77.48311304645532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104995,74,Other,West Anthony,Oklahoma,Yes,No,Card,Dr. Williams,Covid-19,Med_E,Specialist,65,112,169,61,309.16444606680665,183,10,3,63.20478776719894
104996,37,Male,Mckeetown,Utah,Yes,No,Cash,Dr. Jones,Covid-19,Med_C,General,missing,111,165,67,missing,191,47,2,113.86764025370326
104997,51,Female,Lake Lisachester,Nebraska,Yes,Yes,Insurance,Dr. Williams,Allergy,Med_D,Follow-Up,89,118,179,71,320.8304180033907,172,39,3,35.622420232475356
104998,missing,Female,Heidiborough,missing,Yes,No,Cash,Dr. Smith,Covid-19,Med_A,Follow-Up,74,123,167,75,5.341231420036174,180,79,missing,124.4083409893742


## Idenfying missing values in the dataset


Checking how many values are missing in the dataset



In [8]:
HCD_df.isnull().values.sum()

19993

Replacing all missing values with 0

In [9]:
HCD_df = HCD_df.fillna(0)

Confirming there are no missing values

In [10]:
HCD_df.isnull().values.sum()

0

Here I replaced all missing values with 0

In [11]:
HCD_df = HCD_df.replace('missing', 0)

## Removing any duplicate rows and columns

Checking to see if any columns were duplicated, and I found that no columns were duplicated.

In [12]:
HCD_df.columns.duplicated()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False])

I am checking if any rows were duplicated, and found that 2612 rows were duplicated

In [13]:
HCD_df.duplicated().values.sum()

5000

Here I am dropping all rows that were duplicates

In [14]:
HCD_df.drop_duplicates(inplace=True)

I am checking to make sure no duplicates are left

In [15]:
HCD_df.duplicated().values.sum()

0

## Cleaning column names

I listed the names of each column in the dataset

In [16]:
HCD_df.columns

Index(['Patient Age', 'Gender', 'City of Residence', 'State of Residence',
       'Has Insurance', 'Visited Last Month', 'Payment Method',
       'Preferred Doctor', 'Disease Diagnosed', 'Medication Prescribed',
       'Type of Appointment', 'Average Heart Rate', 'Average BP',
       'Height (in cm)', 'Weight (in kg)', 'Payment Due ($)',
       'Last Visit (days ago)', 'Visit Duration (mins)', 'Number of Tests',
       'Prescription Cost ($)'],
      dtype='object')

Here I renamed the columns.
I read that column names should be all lower case and spaces should be defined by underscores, so that is what I did

In [17]:
HCD_df = HCD_df.rename(columns = {'Patient Age':'patient_age',
                     'Gender':'gender',
                     'City of Residence':'city_of_residence',
                     'State of Residence':'state_of_residence',
                     'Has Insurance':'has_insurance',
                     'Visited Last Month':'visited_last_month',
                     'Payment Method':'payment_method',
                     'Preferred Doctor':'preferred_doctor',
                     'Disease Diagnosed':'disease_diagnosed',
                     'Medication Prescribed':'medication_prescribed',
                     'Type of Appointment':'type_of_appointment',
                     'Average Heart Rate':'average_heart_rate',
                     'Average BP':'average_bp',
                     'Height (in cm)':'height_cm',
                     'Weight (in kg)':'weight_kg',
                     'Payment Due ($)':'payment_due',
                     'Last Visit (days ago)':'last_visit_days',
                     'Visit Duration (mins)':'visit_duration_(mins)',
                     'Number of Tests':'number_of_tests',
                     'Prescription Cost ($)':'prescription_cost_($)'})

I am making sure all my column names are renamed

In [18]:
HCD_df.columns

Index(['patient_age', 'gender', 'city_of_residence', 'state_of_residence',
       'has_insurance', 'visited_last_month', 'payment_method',
       'preferred_doctor', 'disease_diagnosed', 'medication_prescribed',
       'type_of_appointment', 'average_heart_rate', 'average_bp', 'height_cm',
       'weight_kg', 'payment_due', 'last_visit_days', 'visit_duration_(mins)',
       'number_of_tests', 'prescription_cost_($)'],
      dtype='object')

## Creating a new column based on existing ones


Checking to see the type of the column average heart rate

In [19]:
HCD_df["average_heart_rate"].dtype

dtype('O')

First I am converting the average heart rate column from object (string) to integer type

In [20]:
HCD_df["average_heart_rate"] = HCD_df["average_heart_rate"].astype('int')

I created a column called healthy_heart_rate that uses the average heart rate to identify if it is a healthy heart rate or not. A healthy heart rate is between 60-100.

If a heart rate is between 60 and 100 it is considered healthy and identified as true in the dataset, if the herat rate is less than 60 or greater than 100 it is considered abnormal and identified as False in the dataset.

In [21]:
HCD_df['healthy_heart_rate'] = np.where(np.logical_and(HCD_df['average_heart_rate']>60, HCD_df['average_heart_rate']<100), True, False)

This is me printing out the new column: healthy_heart_rate

In [22]:
HCD_df['healthy_heart_rate']

0          True
1         False
2          True
3          True
4          True
          ...  
104994     True
104995     True
104996    False
104997     True
104998     True
Name: healthy_heart_rate, Length: 100000, dtype: bool

In [23]:
HCD_df

,patient_age,gender,city_of_residence,state_of_residence,has_insurance,visited_last_month,payment_method,preferred_doctor,disease_diagnosed,medication_prescribed,...,average_heart_rate,average_bp,height_cm,weight_kg,payment_due,last_visit_days,visit_duration_(mins),number_of_tests,prescription_cost_($),healthy_heart_rate
0,45,Other,West Brian,Kentucky,Yes,Yes,Insurance,Dr. Williams,Flu,Med_E,...,66,111,183,70,65.45721578126224,193,92,5,15.71751735532889,True
1,25,Female,East Jocelynfurt,Colorado,No,Yes,Card,Dr. Johnson,Flu,Med_B,...,59,0,174,58,430.68365678679174,195,0,4,80.78647284463952,False
2,51,Other,South Lindseyland,Ohio,Yes,No,Cash,Dr. Williams,Covid-19,Med_D,...,79,119,161,56,315.0709305262176,0,37,2,64.3921393009105,True
3,18,Female,Taylorfort,Pennsylvania,Yes,0,Cash,Dr. Williams,Flu,Med_A,...,99,115,171,52,320.2998987723972,175,108,2,8.872859215315316,True
4,48,Female,Birdmouth,Montana,No,No,Insurance,Dr. Brown,0,Med_C,...,93,89,186,87,264.2147372473319,188,113,3,77.48311304645532,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104994,53,Female,Michelleburgh,South Dakota,No,No,Card,Dr. Williams,Flu,Med_B,...,68,109,172,75,488.0316041186948,167,25,5,69.54228578689846,True
104995,74,Other,West Anthony,Oklahoma,Yes,No,Card,Dr. Williams,Covid-19,Med_E,...,65,112,169,61,309.16444606680665,183,10,3,63.20478776719894,True
104996,37,Male,Mckeetown,Utah,Yes,No,Cash,Dr. Jones,Covid-19,Med_C,...,0,111,165,67,0,191,47,2,113.86764025370326,False
104997,51,Female,Lake Lisachester,Nebraska,Yes,Yes,Insurance,Dr. Williams,Allergy,Med_D,...,89,118,179,71,320.8304180033907,172,39,3,35.622420232475356,True


## Aggregating data using groupby and compute summary statistics

Here I am grouping by'healthy_heart_rate" column and the results are below.

In [24]:
HCD_df.groupby(['healthy_heart_rate']).describe()

average_heart_rate                                          \
                                count       mean        std   min   25%   50%   
healthy_heart_rate                                                              
False                          9930.0  38.447231  42.079148   0.0   0.0   0.0   
True                          90070.0  79.638359   8.692928  61.0  73.0  80.0   

                                 
                     75%    max  
healthy_heart_rate               
False               60.0  120.0  
True                86.0   99.0

Here I am also grouping by "gender" column, and the results are below

In [25]:
HCD_df.groupby(['gender']).describe()

average_heart_rate                                                    
                    count       mean        std  min   25%   50%   75%    max
gender                                                                       
0                  5000.0  75.630200  19.866272  0.0  72.0  79.0  86.0  115.0
Female            31674.0  75.440077  19.970485  0.0  71.0  79.0  86.0  116.0
Male              31728.0  75.450674  19.934438  0.0  71.0  79.0  86.0  120.0
Other             31598.0  75.741155  19.768542  0.0  72.0  79.0  86.0  119.0

Here I converted the weight column into an integer to be able to create a pivot table later on.

In [26]:
HCD_df["weight_kg"] = HCD_df["weight_kg"].astype('int')

Here I converted the height column into an integer to be able to create a pivot table later on.

In [27]:
HCD_df["height_cm"] = HCD_df["height_cm"].astype('int')

Here is my pivot table with index being the gender and the columns used were average heart rate, weight, and height

In [28]:
HCD_df.pivot_table(index=['gender'], values=['average_heart_rate', 'weight_kg', 'height_cm'])

,average_heart_rate,height_cm,weight_kg
gender,,,
0,75.630200,166.350800,65.720600
Female,75.440077,165.602797,66.038139
Male,75.450674,165.755453,66.418904
Other,75.741155,166.022122,66.194190


## Polars


Here I am starting the time for polars

In [29]:
start_time = time.time()

Here I am loading the dataset using polars

In [30]:
pl.read_csv('healthcare_data_cleaning.csv')

Patient Age,Gender,City of Residence,State of Residence,Has Insurance,Visited Last Month,Payment Method,Preferred Doctor,Disease Diagnosed,Medication Prescribed,Type of Appointment,Average Heart Rate,Average BP,Height (in cm),Weight (in kg),Payment Due ($),Last Visit (days ago),Visit Duration (mins),Number of Tests,Prescription Cost ($)
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""45""","""Other""","""West Brian""","""Kentucky""","""Yes""","""Yes""","""Insurance""","""Dr. Williams""","""Flu""","""Med_E""","""General""","""66""","""111""","""183""","""70""","""65.45721578126…","""193""","""92""","""5""","""15.71751735532…"
"""25""","""Female""","""East Jocelynfu…","""Colorado""","""No""","""Yes""","""Card""","""Dr. Johnson""","""Flu""","""Med_B""","""Specialist""","""59""","""missing""","""174""","""58""","""430.6836567867…","""195""","""missing""","""4""","""80.78647284463…"
"""51""","""Other""","""South Lindseyl…","""Ohio""","""Yes""","""No""","""Cash""","""Dr. Williams""","""Covid-19""","""Med_D""","""General""","""79""","""119""","""161""","""56""","""315.0709305262…","""missing""","""37""","""2""","""64.39213930091…"
"""18""","""Female""","""Taylorfort""","""Pennsylvania""","""Yes""","""missing""","""Cash""","""Dr. Williams""","""Flu""","""Med_A""","""General""","""99""","""115""","""171""","""52""","""320.2998987723…","""175""","""108""","""2""","""8.872859215315…"
"""48""","""Female""","""Birdmouth""","""Montana""","""No""","""No""","""Insurance""","""Dr. Brown""","""None""","""Med_C""","""Specialist""","""93""","""89""","""186""","""87""","""264.2147372473…","""188""","""113""","""3""","""77.48311304645…"
"""62""","""missing""","""North Donald""","""Oklahoma""","""No""","""Yes""","""Card""","""Dr. Jones""","""Covid-19""","""Med_B""","""General""","""89""","""118""","""185""","""88""","""207.6380381372…","""185""","""101""","""2""","""38.56558697880…"
"""52""","""Male""","""Lake Jennifer""","""missing""","""No""","""No""","""Cash""","""missing""","""Covid-19""","""Med_A""","""Emergency""","""76""","""114""","""155""","""70""","""missing""","""187""","""41""","""4""","""85.48049622708…"
"""52""","""Male""","""Vanessaburgh""","""Connecticut""","""Yes""","""Yes""","""Insurance""","""Dr. Williams""","""Flu""","""Med_E""","""General""","""99""","""129""","""170""","""47""","""387.0850435350…","""179""","""108""","""2""","""76.96592632951…"
"""20""","""Female""","""West Briannabu…","""Montana""","""No""","""Yes""","""Card""","""Dr. Jones""","""Flu""","""Med_C""","""Specialist""","""67""","""108""","""173""","""86""","""172.5483427968…","""174""","""91""","""2""","""142.3171076789…"


This is subtracting the current time with the start time to get the time for polars to load the dataset.

In [31]:
polars_time = time.time() - start_time

Here I am starting the time for Modin

In [32]:
start_time = time.time()

Here I am loading the dataset using Modin

In [33]:
mpd.read_csv("healthcare_data_cleaning.csv")

To remove this warning, run the following python code before doing dataframe operations:

    from distributed import Client

    client = Client()

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:43983
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:35147'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:44785'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:40961', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:40961
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:48414
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:33909', name: 

,Patient Age,Gender,City of Residence,State of Residence,Has Insurance,Visited Last Month,Payment Method,Preferred Doctor,Disease Diagnosed,Medication Prescribed,Type of Appointment,Average Heart Rate,Average BP,Height (in cm),Weight (in kg),Payment Due ($),Last Visit (days ago),Visit Duration (mins),Number of Tests,Prescription Cost ($)
0,45,Other,West Brian,Kentucky,Yes,Yes,Insurance,Dr. Williams,Flu,Med_E,General,66,111,183,70,65.45721578126224,193,92,5,15.71751735532889
1,25,Female,East Jocelynfurt,Colorado,No,Yes,Card,Dr. Johnson,Flu,Med_B,Specialist,59,missing,174,58,430.68365678679174,195,missing,4,80.78647284463952
2,51,Other,South Lindseyland,Ohio,Yes,No,Cash,Dr. Williams,Covid-19,Med_D,General,79,119,161,56,315.0709305262176,missing,37,2,64.3921393009105
3,18,Female,Taylorfort,Pennsylvania,Yes,missing,Cash,Dr. Williams,Flu,Med_A,General,99,115,171,52,320.2998987723972,175,108,2,8.872859215315316
4,48,Female,Birdmouth,Montana,No,No,Insurance,Dr. Brown,NaN,Med_C,Specialist,93,89,186,87,264.2147372473319,188,113,3,77.48311304645532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104995,74,Other,West Anthony,Oklahoma,Yes,No,Card,Dr. Williams,Covid-19,Med_E,Specialist,65,112,169,61,309.16444606680665,183,10,3,63.20478776719894
104996,37,Male,Mckeetown,Utah,Yes,No,Cash,Dr. Jones,Covid-19,Med_C,General,missing,111,165,67,missing,191,47,2,113.86764025370326
104997,51,Female,Lake Lisachester,Nebraska,Yes,Yes,Insurance,Dr. Williams,Allergy,Med_D,Follow-Up,89,118,179,71,320.8304180033907,172,39,3,35.622420232475356
104998,missing,Female,Heidiborough,missing,Yes,No,Cash,Dr. Smith,Covid-19,Med_A,Follow-Up,74,123,167,75,5.341231420036174,180,79,missing,124.4083409893742


This is subtracting the current time with the start time to get the time for modin to load the dataset.

In [34]:
modin_time = time.time() - start_time

In [35]:
print("Modin time : ", modin_time)

Modin time :  9.437889337539673


In [36]:
print("Polars time : ", polars_time)

Polars time :  0.3036491870880127


I observed that polars is faster then modin when it comes to loading datasets.